Traning neural network to transform positive yelp reviews to negative and vice versa

In [1]:
!pip install ipywidgets
!pip install accelerate
!pip install pfrl@git+https://github.com/voidful/pfrl.git
!pip install textrl
#instal nltk
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 138 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 198 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 3.2 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 219 kB 2.4 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/voidful/pfrl.git to /tmp/pip-install-mrybftt2/pfrl_44f4a0f67f51440eb6f1b1ed0b828977
  Running command git clone -q https://github.com/voidful/pfrl.git /tmp/pip-install-mrybftt2/pfrl_44f4a0f67f51440eb6f1b1ed0b828977
  Resolved https://github.com/voidful/pfrl.git to commit 2ad3d51a7a971f3fe7f2

In [156]:
from sklearn.utils import shuffle

#read training data from yelp_data folder
#test.0 contains negative yelp reviews
#refrence0-3.0 contains positive yelp reviews with same context as test.0

#test.1 contains positive yelp reviews
#refrence0-3.1 contains negative yelp reviews with same context as test.1

#load paraller datataset
rewiev_files = ["./DualRLStyleTransfer/yelp_data/test.0",
                "./DualRLStyleTransfer/yelp_data/test.1",
                "./DualRLStyleTransfer/yelp_data/dev.0",
                "./DualRLStyleTransfer/yelp_data/dev.1",
                "./DualRLStyleTransfer/yelp_data/train.0",
                "./DualRLStyleTransfer/yelp_data/train.1"]
reference_files = ["./DualRLStyleTransfer/yelp_refrence/reference0.0",
                  "./DualRLStyleTransfer/yelp_refrence/reference0.1",
                  "./DualRLStyleTransfer/yelp_refrence/reference1.0",
                  "./DualRLStyleTransfer/yelp_refrence/reference1.1",
                    "./DualRLStyleTransfer/yelp_refrence/reference2.0",
                    "./DualRLStyleTransfer/yelp_refrence/reference2.1",
                  "./DualRLStyleTransfer/yelp_refrence/reference3.0",
                  "./DualRLStyleTransfer/yelp_refrence/reference3.1"]

references_to_test = ([],[])
for i in range(0, len(reference_files)):
    print(reference_files[i])
    with open(reference_files[i]) as f:
        #read line by line
        lines = f.readlines()
        #remove \n from end of line
        lines = [x.strip() for x in lines]
        #remove spaces before , . ! ?
        lines = [x.strip() for x in lines]
        lines = [x.replace(" .",".") for x in lines]
        lines = [x.replace(" !","!") for x in lines]
        lines = [x.replace(" ?","?") for x in lines]
        lines = [x.replace(" '","'") for x in lines]
        lines = [x.replace(' ,',',') for x in lines]
        lines = [x.replace(' )',')') for x in lines]
        lines = [x.replace(' :',':') for x in lines]
        lines = [x.replace('( ','(') for x in lines]
        lines = [x.replace(' - ','-') for x in lines]
        lines = [x.replace('$ _num_','$_num_') for x in lines]
        lines = [x.replace('_num_ $','_num_$') for x in lines]
        lines = [x.replace(' - ','-') for x in lines]
        references_to_test[0].extend(lines)
        references_to_test[1].extend([i%2]*len(lines))
#train classifier if the rewiev is positive or negative
#classifier is used to calculate reward
train_data_XY = ([],[])
test_data_XY = ([],[])
dev_data_XY = ([],[])


for i in range(0, len(rewiev_files)):
    print(rewiev_files[i])
    with open(rewiev_files[i]) as f:
        lines = f.readlines()
        #remove \n from end of line
        lines = [x.strip() for x in lines]
        lines = [x.replace(" .",".") for x in lines]
        lines = [x.replace(" !","!") for x in lines]
        lines = [x.replace(" ?","?") for x in lines]
        lines = [x.replace(" '","'") for x in lines]
        lines = [x.replace(' ,',',') for x in lines]
        lines = [x.replace(' )',')') for x in lines]
        lines = [x.replace(' :',':') for x in lines]
        lines = [x.replace('( ','(') for x in lines]
        lines = [x.replace(' - ','-') for x in lines]
        lines = [x.replace('$ _num_','$_num_') for x in lines]
        lines = [x.replace('_num_ $','_num_$') for x in lines]
        #create sentiment list same size as lines
        sentiment = [i%2] * len(lines)
        #add lines and sentiment to train_data_XY array
        if i > 3:
            train_data_XY[0].extend(lines)
            train_data_XY[1].extend(sentiment)
        elif i > 1:
            dev_data_XY[0].extend(lines)
            dev_data_XY[1].extend(sentiment)
        else:
            test_data_XY[0].extend(lines)
            test_data_XY[1].extend(sentiment)

#shuffle data together so that positive and negative rewiews are mixed

indexes = shuffle(range(0,len(test_data_XY[0])))
test_data_XY = (list(test_data_XY[0][i] for i in indexes),list(test_data_XY[1][i] for i in indexes))
test_len = len(test_data_XY[0])
index = indexes.copy()
for i in range(1, int(len(references_to_test[0])/test_len)):
    #extende the indexes by adding new indexes ofsset by test_len
    indexes.extend([x + test_len*i for x in index])
print(len(indexes), len(references_to_test[0]),test_len)
references_to_test = (list(references_to_test[0][i] for i in indexes),list(references_to_test[1][i] for i in indexes))
indexes = shuffle(range(0,len(dev_data_XY[0])))
dev_data_XY = (list(dev_data_XY[0][i] for i in indexes),list(dev_data_XY[1][i] for i in indexes))
indexes = shuffle(range(0,len(train_data_XY[0])))
train_data_XY = (list(train_data_XY[0][i] for i in indexes),list(train_data_XY[1][i] for i in indexes))

#print first 3 rewievs
print("test_data_XY")
for i in range(0,10):
    print(test_data_XY[0][i] + "  " + str(test_data_XY[1][i]) + " --> " + references_to_test[0][i+test_len*2])


./DualRLStyleTransfer/yelp_refrence/reference0.0
./DualRLStyleTransfer/yelp_refrence/reference0.1
./DualRLStyleTransfer/yelp_refrence/reference1.0
./DualRLStyleTransfer/yelp_refrence/reference1.1
./DualRLStyleTransfer/yelp_refrence/reference2.0
./DualRLStyleTransfer/yelp_refrence/reference2.1
./DualRLStyleTransfer/yelp_refrence/reference3.0
./DualRLStyleTransfer/yelp_refrence/reference3.1
./DualRLStyleTransfer/yelp_data/test.0
./DualRLStyleTransfer/yelp_data/test.1
./DualRLStyleTransfer/yelp_data/dev.0
./DualRLStyleTransfer/yelp_data/dev.1
./DualRLStyleTransfer/yelp_data/train.0
./DualRLStyleTransfer/yelp_data/train.1
4000 4000 1000
test_data_XY
however, everyone else paid after their nails were completed.  0 --> everyone else paid after they have a good meal
there was no personality to our server.  0 --> there was huge personality to our serve.
if your patio is south facing, you're out of luck.  0 --> if your patio is south facing, you're in luck
this is the best seafood joint in town

In [157]:
import regex
alfanumerical_and_punctination = regex.compile('[^a-zA-Z0-9\s\.\,\;\:\!\?\-\'\"]')
#Build a language model for the yelp dataset
#vocab has monogtams arranged by frequency in the dataset (most frequent first)
with open('./DualRLStyleTransfer/yelp_data/vocab') as f:
    vocab = f.readlines()
    #remove \n from end of line
    vocab = [x.strip() for x in vocab]

#how well do the ngrams fit the vocab of the dataset
def calaculate_ngrams_matching_vocab_category(ngrams, vocab):
    probability = 1
    for ngram in ngrams:
        if ngram in vocab:
            probability += vocab.index(ngram)
        else:
            probability += len(vocab)
    return 1/ (probability/len(vocab))

def split_string_into_ngrams(string, n):
    global alfanumerical
    #convert to lower case
    string = string.lower()
    #remove special characters (keep only letters, numbers and spaces)
    string = alfanumerical_and_punctination.sub('', string)
    #replace space with <blank>
    string = string.replace(" ", " <blank> ")
    #raplace numbers with _numb_
    string = regex.sub(r'\d+', '_numb_', string)
    #split punctuation from words (add space before punctuation . , ! ?)
    string = string.replace(".", " .")
    string = string.replace(",", " ,")
    string = string.replace("!", " !")
    string = string.replace("?", " ?")
    #split string into words
    words = string.split()
    #add start and end tokens
    words.insert(0, "<s>")
    words.append("</s>")
    ngrams = [words[i:i + n] for i in range(len(words) - n + 1)]
    return words

def ngrams_to_string(ngrams):
    string = ""
    for word in ngrams:
        if word == "<blank>":
            string += " "
        elif word != "<s>" and word != "</s>":
            string += word
    return string

test1 = "The service was terrible. I will never go there again"
test2 = "The missile knows where it is, because it knows where it isn't"
ngrams = split_string_into_ngrams(test1, 1)
print(ngrams, ngrams_to_string(ngrams))
print(calaculate_ngrams_matching_vocab_category(ngrams, vocab))
ngrams = split_string_into_ngrams(test2, 1)
print(ngrams, ngrams_to_string(ngrams))
print(calaculate_ngrams_matching_vocab_category(ngrams, vocab))

['<s>', 'the', '<blank>', 'service', '<blank>', 'was', '<blank>', 'terrible', '.', '<blank>', 'i', '<blank>', 'will', '<blank>', 'never', '<blank>', 'go', '<blank>', 'there', '<blank>', 'again', '</s>'] the service was terrible. i will never go there again
16.7
['<s>', 'the', '<blank>', 'missile', '<blank>', 'knows', '<blank>', 'where', '<blank>', 'it', '<blank>', 'is', ',', '<blank>', 'because', '<blank>', 'it', '<blank>', 'knows', '<blank>', 'where', '<blank>', 'it', '<blank>', "isn't", '</s>'] the missile knows where it is, because it knows where it isn't
0.43780721876316653


In [222]:
#use the english language model to calculate the probability of a sentence
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.lm import Vocabulary
from nltk.util import ngrams
import numpy as np

def calculate_sentence_probability(sentence, n, vocab):
    #split sentence into ngrams
    ngrams = split_string_into_ngrams(sentence, n)
    #create vocab
    vocab = Vocabulary(ngrams, unk_cutoff=1, unk_label="<unk>")
    #create ngram model
    model = MLE(n)
    #train model
    model.fit([ngrams], vocab)
    #calculate probability
    probability = model.perplexity([ngrams])
    return probability



In [182]:
#train classifier with train_data_XY
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import pickle

#train classifier
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                        ('clf', LogisticRegression(random_state=0, max_iter=1000)),
                        ])
text_clf.fit(train_data_XY[0], train_data_XY[1])

#test classifier
predicted = text_clf.predict(test_data_XY[0])
print("accuracy_score: " + str(accuracy_score(test_data_XY[1], predicted)))
print(classification_report(test_data_XY[1], predicted))

#save classifier to tmp folder
with open('./DualRLStyleTransfer/tmp/classifier.pkl', 'wb') as f:
    pickle.dump(text_clf, f)

#load classifier from tmp folder
with open('./DualRLStyleTransfer/tmp/classifier.pkl', 'rb') as f:
    text_clf = pickle.load(f)


accuracy_score: 0.957
              precision    recall  f1-score   support

           0       0.96      0.95      0.96       500
           1       0.95      0.96      0.96       500

    accuracy                           0.96      1000
   macro avg       0.96      0.96      0.96      1000
weighted avg       0.96      0.96      0.96      1000



## BELU score
BELU or Bilingual Evaluation Understudy is a metric for evaluating the quality of text which has been machine-translated from one natural language to another. We will use this metric to evaluate the quality of our generated reviews. The higher the BELU score, the better the quality of the generated text. 

In [210]:
#BELU score compare test_data_XY and refrence_files
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

#we can also use our own tokenizer (split_string_into_ngrams) gives similar score
hf_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

refrences = []
i = 16
for n in range(0,int(len(references_to_test[0])/len(test_data_XY[0]))):
    refrences.append(references_to_test[0][n*len(test_data_XY[0]) + i])
hypothesis = test_data_XY[0][i]
print(refrences)
print(hypothesis)

def calculate_bleu_score(hypothesis, refrences):
    ngrams = hf_tokenizer.tokenize(hypothesis) #split_string_into_ngrams(hypothesis, 1)
    #print(ngrams)
    ref = []
    for n in range(0,len(refrences)):
        ref.append(hf_tokenizer.tokenize(refrences[n]))
    #print(ref)
    #calculate BLEU score for ngrams
    return sentence_bleu(ref, ngrams, weights=(1.0, 0.0, 0.0, 0.0), smoothing_function=SmoothingFunction().method1)

#calculate BLEU score for hypothesis and refrences
print("BLEU score for hypothesis and refrences: " + str(calculate_bleu_score(hypothesis, refrences)))

['you can not pay me to not ever shop here!', 'you do not need to pay me for shopping here!', 'i will always go shopping here!', 'you can pay me to shop there every day']
you can not pay me to ever shop here!
BLEU score for hypothesis and refrences: 1.0


In [211]:
#calculate BLEU score for the test dataset (this should give us a high result becuse the data was manualy curated to be similar to the refrences)
bleu_score = 0
for i in range(0,len(test_data_XY[0])):
    refrences = []
    for n in range(0,int(len(references_to_test[0])/len(test_data_XY[0]))):
        refrences.append(references_to_test[0][n*len(test_data_XY[0]) + i])
    hypothesis = test_data_XY[0][i]
    bleu_score += calculate_bleu_score(hypothesis, refrences)
bleu_score /= len(test_data_XY[0])
print("BLEU score for the test dataset: " + str(bleu_score))

BLEU score for the test dataset: 0.8500289256073363


In [212]:
#evaluate generated sentence by combining classifier score, BELU score, category matching score
def evaluate_sentence(sentence, classifier, vocab, references, target_category):
    #calculate classifier score
    classifier_score = classifier.predict_proba([sentence])[0][target_category]
    #calculate BLEU score
    bleu_score = calculate_bleu_score(sentence, references)
    #calculate category matching score
    ngrams = split_string_into_ngrams(sentence, 1)
    category_matching_score = calaculate_ngrams_matching_vocab_category(ngrams, vocab)
    #calculate final score
    score = classifier_score + bleu_score
    return score/2, classifier_score, bleu_score, category_matching_score

In [213]:
#test evaluate_sentence
for i in range(0,len(test_data_XY[0])):
    refrences = []
    for n in range(0,int(len(references_to_test[0])/len(test_data_XY[0]))):
        refrences.append(references_to_test[0][n*len(test_data_XY[0]) + i])
    hypothesis = test_data_XY[0][i]
    score, clasifier_score, BELU_socre, category_score = evaluate_sentence(hypothesis, text_clf, vocab, refrences, test_data_XY[1][i])
    print("score: " + str(score) + " clasifier_score: " + str(clasifier_score) + "-->" + str(test_data_XY[1][i]) + " BELU_score: " + str(BELU_socre), ":" + hypothesis)

score: 0.6554538031855082 clasifier_score: 0.9472712427346527-->0 BELU_score: 0.36363636363636365 :however, everyone else paid after their nails were completed.
score: 0.9365984346787388 clasifier_score: 0.9981968693574776-->0 BELU_score: 0.875 :there was no personality to our server.
score: 0.796107134792198 clasifier_score: 0.7350714124415387-->0 BELU_score: 0.8571428571428571 :if your patio is south facing, you're out of luck.
score: 0.9992065909837446 clasifier_score: 0.9984131819674894-->1 BELU_score: 1.0 :this is the best seafood joint in town.
score: 0.8852615611080441 clasifier_score: 0.8538564555494217-->0 BELU_score: 0.9166666666666666 :there is a reason they can get you in fairly quickly.
score: 0.9999909210238291 clasifier_score: 0.9999818420476582-->1 BELU_score: 1.0 :i spent time with my best buds and enjoyed some excellent wine and food.
score: 0.8713730448806022 clasifier_score: 0.9927460897612044-->0 BELU_score: 0.75 :needless to say, i will not be returning to this pl

In [215]:
import pfrl
from textrl import TextRLEnv, TextRLActor, train_agent_with_evaluation
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSequenceClassification
import logging
import sys

In [216]:
#use tokenizer to split string into ngrams
encoded = tokenizer.encode("This is a test sentence. Using only neural networks is a bad idea.")
print(encoded, tokenizer.decode(encoded))

[1212, 318, 257, 1332, 6827, 13, 8554, 691, 17019, 7686, 318, 257, 2089, 2126, 13] This is a test sentence. Using only neural networks is a bad idea.


In [218]:
#testing hugingface sentiment analysis
hf_model_name = "distilbert-base-uncased-finetuned-sst-2-english"
hf_model = AutoModelForSequenceClassification.from_pretrained(hf_model_name)
hf_tokenizer = AutoTokenizer.from_pretrained(hf_model_name)
hf_classifier = pipeline('sentiment-analysis', model=hf_model, tokenizer=hf_tokenizer)

#tokenizer testing
sequence = "This is a test sentence. Using only neural networks is a bad idea."
tokens = hf_tokenizer.tokenize(sequence)
print(tokens)
ids = hf_tokenizer.convert_tokens_to_ids(tokens)
print(ids)
decoded = hf_tokenizer.decode(ids)
print(decoded)
#print(hf_classifier('We are very happy to show you the Transformers library.'))


['this', 'is', 'a', 'test', 'sentence', '.', 'using', 'only', 'neural', 'networks', 'is', 'a', 'bad', 'idea', '.']
[2023, 2003, 1037, 3231, 6251, 1012, 2478, 2069, 15756, 6125, 2003, 1037, 2919, 2801, 1012]
this is a test sentence. using only neural networks is a bad idea.


In [219]:
#testing sentiment analysis accuracy on the test dataset
accuracy = 0
for i in range(0,len(test_data_XY[0])):
    res = hf_classifier(test_data_XY[0][i])
    if res[0]['label'] == 'POSITIVE' and test_data_XY[1][i] == 1:
        accuracy += 1
    elif res[0]['label'] == 'NEGATIVE' and test_data_XY[1][i] == 0:
        accuracy += 1
    #print(res, test_data_XY[1][i], test_data_XY[0][i])
accuracy /= len(test_data_XY[0])
print("accuracy: " + str(accuracy))

accuracy: 0.927


In [221]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text
    
